In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Changing the transformers and tokenizer versions

In [2]:
pip install tokenizers==0.8.0rc4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.18.0 requires tokenizers!=0.11.3,<0.13,>=0.11.1, but you have tokenizers 0.8.0rc4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 KB 3.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 3.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import transformers
transformers.__version__

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


'3.0.1'

In [5]:
import tokenizers
tokenizers.__version__

'0.8.0.rc4'

### Imports

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification,AutoConfig

## Reading data

In [8]:
un_neg_data= pd.read_csv("../input/nertag/unnegated_File_Name.csv")

In [9]:
un_neg_data

,Sentence,Tags
0,I am looking for a black gloss 33 inch firecla...,"O,O,O,O,O,B-COLOUR,B-TEXTURE,B-SIZE,I-SIZE,B-P..."
1,Looking for pre workout Pump addict instead of...,"O,O,B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,O,..."
2,i need a 48 inch glass sliding goof and a show...,"O,O,O,B-SIZE,I-SIZE,B-MATERIAL,B-PRODUCT,I-PRO..."
3,"Hello, do any of your free standing tubs have ...","O,O,O,O,O,B-ATTRIBUTE,I-ATTRIBUTE,B-PRODUCT,O,..."
4,I'm looking for a 24 inch white mirror that is...,"O,O,O,O,B-SIZE,I-SIZE,B-COLOUR,B-PRODUCT,O,O,B..."
...,...,...
162,What rectangular shower units are available?,"O,B-SHAPE,B-PRODUCT,I-PRODUCT,O,O"
163,I am looking for a 35 inch bathroom sink count...,"O,O,O,O,O,B-SIZE,I-SIZE,B-PRODUCT,I-PRODUCT,I-..."
164,I'm looking f or a Black Matte bathtub double ...,"O,O,O,O,O,B-COLOUR,B-TEXTURE,B-PRODUCT,I-PRODU..."
165,Need a 27 inch frameless shower door,"O,O,B-SIZE,I-SIZE,B-ATTRIBUTE,B-PRODUCT,I-PRODUCT"


In [7]:

data = pd.read_csv("../input/nertag/File_Name.csv")

In [8]:
data

,Sentence,Tags
0,I am looking for a black gloss 33 inch firecla...,"O,O,O,O,O,B-COLOUR,B-TEXTURE,B-SIZE,I-SIZE,B-P..."
1,Looking for pre workout Pump addict instead of...,"O,O,B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,O,..."
2,i need a 48 inch glass sliding goof and a show...,"O,O,O,B-SIZE,I-SIZE,B-MATERIAL,B-PRODUCT,I-PRO..."
3,"Hello, do any of your free standing tubs have ...","O,O,O,O,O,B-ATTRIBUTE,I-ATTRIBUTE,B-PRODUCT,O,..."
4,I'm looking for a 24 inch white mirror that is...,"O,O,O,O,B-SIZE,I-SIZE,B-COLOUR,B-PRODUCT,O,O,B..."
...,...,...
162,What rectangular shower units are available?,"O,B-SHAPE,B-PRODUCT,I-PRODUCT,O,O"
163,I am looking for a 35 inch bathroom sink count...,"O,O,O,O,O,B-SIZE,I-SIZE,B-PRODUCT,I-PRODUCT,I-..."
164,I'm looking f or a Black Matte bathtub double ...,"O,O,O,O,O,B-COLOUR,B-TEXTURE,B-PRODUCT,I-PRODU..."
165,Need a 27 inch frameless shower door,"O,O,B-SIZE,I-SIZE,B-N-ATTRIBUTE,B-PRODUCT,I-PR..."


In [69]:
secondary_data = pd.read_csv("../input/nertag/clean_data.csv")

In [10]:
unneg_secondary_data= pd.read_csv("../input/nertag/un-negated_clean_data.csv")

In [11]:
unneg_secondary_data

,Tokens,Tags,is_negative,sentence
0,I,O,False,0
1,am,O,False,0
2,looking,O,False,0
3,for,O,False,0
4,a,O,False,0
...,...,...,...,...
2345,30,B-SIZE,False,166
2346,inch,I-SIZE,False,166
2347,white,B-COLOUR,False,166
2348,desk,B-PRODUCT,False,166


In [12]:
unneg_secondary_data['Sentence']= unneg_secondary_data[['sentence','Tokens','is_negative']].groupby(['sentence'])['Tokens'].transform(lambda x: ' '.join(x))

In [13]:
unneg_secondary_data['is_negative'] = unneg_secondary_data['is_negative'].map({True: 'True', False: 'False'})

In [14]:
unneg_secondary_data['is_negative']= unneg_secondary_data[['sentence','Tokens','is_negative']].groupby(['sentence'])['is_negative'].transform(lambda x: ','.join(x))

In [15]:
unneg_secondary_data

,Tokens,Tags,is_negative,sentence,Sentence
0,I,O,"False,False,False,False,False,False,False,Fals...",0,I am looking for a black gloss 33 inch firecla...
1,am,O,"False,False,False,False,False,False,False,Fals...",0,I am looking for a black gloss 33 inch firecla...
2,looking,O,"False,False,False,False,False,False,False,Fals...",0,I am looking for a black gloss 33 inch firecla...
3,for,O,"False,False,False,False,False,False,False,Fals...",0,I am looking for a black gloss 33 inch firecla...
4,a,O,"False,False,False,False,False,False,False,Fals...",0,I am looking for a black gloss 33 inch firecla...
...,...,...,...,...,...
2345,30,B-SIZE,"False,False,False,False,False,False,False",166,Looking for 30 inch white desk set
2346,inch,I-SIZE,"False,False,False,False,False,False,False",166,Looking for 30 inch white desk set
2347,white,B-COLOUR,"False,False,False,False,False,False,False",166,Looking for 30 inch white desk set
2348,desk,B-PRODUCT,"False,False,False,False,False,False,False",166,Looking for 30 inch white desk set


In [16]:
unneg_secondary_data = unneg_secondary_data[['Sentence','is_negative']]

In [17]:
unneg_secondary_data = unneg_secondary_data.drop_duplicates().reset_index(drop=True)

In [18]:
unneg_secondary_data

,Sentence,is_negative
0,I am looking for a black gloss 33 inch firecla...,"False,False,False,False,False,False,False,Fals..."
1,Looking for pre workout Pump addict instead of...,"False,False,False,False,False,False,False,Fals..."
2,i need a 48 inch glass sliding goof and a show...,"False,False,False,False,False,False,False,Fals..."
3,"Hello, do any of your free standing tubs have ...","False,False,False,False,False,False,False,Fals..."
4,I'm looking for a 24 inch white mirror that is...,"False,False,False,False,False,False,False,Fals..."
...,...,...
162,What rectangular shower units are available?,"False,False,False,False,False,False"
163,I am looking for a 35 inch bathroom sink count...,"False,False,False,False,False,False,False,Fals..."
164,I'm looking f or a Black Matte bathtub double ...,"False,False,False,False,False,False,False,Fals..."
165,Need a 27 inch frameless shower door,"False,False,False,False,False,False,False"


In [19]:
unneg_secondary_data.to_csv(r'./unnegated_Fil.csv', index = False)

In [10]:
secondary_data

,Tokens,Tags,sentence
0,I,O,0
1,am,O,0
2,looking,O,0
3,for,O,0
4,a,O,0
...,...,...,...
2345,30,B-SIZE,166
2346,inch,I-SIZE,166
2347,white,B-COLOUR,166
2348,desk,B-PRODUCT,166


### Tokenization

In [24]:
labels_to_ids = {k: v for v, k in enumerate(unneg_secondary_data.Tags.unique())}
ids_to_labels = {v: k for v, k in enumerate(unneg_secondary_data.Tags.unique())}
labels_to_ids

AttributeError: 'DataFrame' object has no attribute 'Tags'

In [20]:
labels_to_ids = {'True':1,'False':0}
ids_to_labels = {1: 'True', 0: 'False'}

In [21]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 128
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [25]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.Sentence[index].strip().split()  
        word_labels = self.data.is_negative[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_pretokenized=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER",use_fast=True)
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

In [26]:
train_size = 0.8
train_dataset = unneg_secondary_data.sample(frac=train_size,random_state=200)
test_dataset = unneg_secondary_data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(unneg_secondary_data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (167, 2)
TRAIN Dataset: (134, 2)
TEST Dataset: (33, 2)


In [27]:
training_set[0]

{'input_ids': tensor([  101,  1045,  2215,  2074,  2023,  5239,  2302,  1996,  2327,  2004,
          1045,  2215,  2327,  5614, 22739,  2006,  1037,  3302,  9844,  2026,
          3829,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [28]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
i           0
want        0
just        0
this        0
cabinet     0
without     0
the         0
top         1
as          0
i           0
want        0
top         0
mounted     0
basins      0
on          0
a           0
surface     0
matching    0
my          0
kitchen     0
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD] 

In [29]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [30]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


### Model Generation

In [37]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# from transformers import BertForSequenceClassification, BertConfig

# config = AutoConfig.from_pretrained("dslim/bert-base-NER")
# config.num_labels = 28
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", config =config)
# model.parameters

In [38]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.6111, device='cuda:0', grad_fn=<NllLossBackward0>)

In [39]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 2])

In [40]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [41]:

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [42]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.6267070174217224
Training loss epoch: 0.5953530967235565
Training accuracy epoch: 0.7660238770585496


In [43]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [44]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.4822540879249573
Validation Loss: 0.4697296777192284
Validation Accuracy: 0.930022945139386


In [46]:
# from seqeval.metrics import classification_report
from sklearn.metrics import f1_score, classification_report

print(classification_report(labels, predictions, labels = list(labels_to_ids.keys()) ))

              precision    recall  f1-score   support

        True       0.20      0.04      0.06        26
       False       0.95      0.99      0.97       451

    accuracy                           0.94       477
   macro avg       0.57      0.51      0.52       477
weighted avg       0.91      0.94      0.92       477

